In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# Import our securitization comparison package
from securitization_comparison import (
    load_fannie_mae,
    TraditionalModel,
    BlockchainModel,
    plot_scores,
    FACTORS
)
from securitization_comparison.viz import create_summary_table

print("✅ All imports successful!")


In [ ]:
# Load loan data (use small sample for demo)
print("Loading loan data...")
try:
    df = load_fannie_mae(sample_size=5000)
    print(f"📊 Loaded {len(df):,} loans with {len(df.columns)} features")
except FileNotFoundError:
    print("📝 No Fannie Mae data found. Creating synthetic test data...")
    # Create synthetic data for demo
    np.random.seed(42)
    n_loans = 5000
    df = pd.DataFrame({
        'loan_id': [f'DEMO_{i:06d}' for i in range(n_loans)],
        'orig_bal': np.random.lognormal(mean=12, sigma=0.5, size=n_loans),
        'orig_rate': np.random.normal(4.5, 1.0, n_loans).clip(2, 10),
        'orig_term': np.random.choice([180, 240, 300, 360], n_loans),
        'state': np.random.choice(['CA', 'TX', 'NY', 'FL', 'IL'], n_loans),
        'credit_score': np.random.normal(720, 50, n_loans).clip(300, 850),
        'dti': np.random.normal(25, 10, n_loans).clip(5, 60),
        'channel': np.random.choice(['R', 'B', 'C'], n_loans)
    })
    print(f"📊 Created synthetic dataset with {len(df):,} loans")

# Display basic info
df.head()


In [ ]:
# Initialize and compare models
print("🏗️ Initializing securitization models...")

# Create models
traditional_model = TraditionalModel(df)
blockchain_model = BlockchainModel(df)

# Compute scores
print("🧮 Computing scores...")
traditional_scores = traditional_model.compute_scores()
blockchain_scores = blockchain_model.compute_scores()

# Create comparison
comparison_df = pd.DataFrame({
    'Traditional': traditional_scores,
    'Blockchain': blockchain_scores
})
comparison_df['Difference'] = comparison_df['Blockchain'] - comparison_df['Traditional']
comparison_df['Winner'] = comparison_df['Difference'].apply(
    lambda x: 'Blockchain' if x > 0 else 'Traditional' if x < 0 else 'Tie'
)

print("\n📊 Score Comparison:")
print(comparison_df.round(1))

# Overall results
total_traditional = comparison_df['Traditional'].sum()
total_blockchain = comparison_df['Blockchain'].sum()
total_difference = total_blockchain - total_traditional
overall_winner = "Blockchain" if total_difference > 0 else "Traditional" if total_difference < 0 else "Tie"

print(f"\n🏆 Overall Winner: {overall_winner}")
print(f"Total Difference: {total_difference:+.1f} points")


In [ ]:
# Create visualizations
print("📊 Creating visualizations...")

# Use built-in plotting function
fig_mpl, fig_plotly = plot_scores(
    traditional_scores, 
    blockchain_scores,
    show_plots=True,
    save_plots=False  # Don't save in demo
)

# Simple bar chart comparison
plt.figure(figsize=(12, 6))
x = np.arange(len(FACTORS))
width = 0.35

trad_values = [traditional_scores[factor] for factor in FACTORS]
chain_values = [blockchain_scores[factor] for factor in FACTORS]

plt.bar(x - width/2, trad_values, width, label='Traditional', alpha=0.8, color='#2E86AB')
plt.bar(x + width/2, chain_values, width, label='Blockchain', alpha=0.8, color='#A23B72')

plt.xlabel('Factors')
plt.ylabel('Score (0-100)')
plt.title('Securitization Comparison: Traditional vs Blockchain')
plt.xticks(x, FACTORS, rotation=45, ha='right')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("✅ Demo complete! See README.md for more details.")
